In [4]:
import requests as r
import pandas as pd
import shutil
import os
import chardet
from zipfile import ZipFile

dev = False

tmp_dir = 'julienne_data/'
start = 2012
end = 2018
ipeds_url = 'https://nces.ed.gov/ipeds/datacenter/data/'
url_pairs = [('HD', ''),
             ('IC', ''),
             ('IC', '_AY'),
             ('EF', 'A_DIST'),
             ('SAL', '_IS'),
             ('OM', '')]

if os.path.isdir(tmp_dir) and not dev:
    shutil.rmtree(tmp_dir)

if not (dev and os.path.isdir(tmp_dir)):
    os.mkdir(tmp_dir)

In [6]:
# Dataframes to be populated and merged per year: hd_df, ic_df, icay_df, efdist_df, salis_df, om_df

for year in range(start, end):
    df = {}
    for prefix, suffix in url_pairs:
        # Skip missing datasets
        print(year + prefix + suffix)
        if prefix == 'OM' and year < 2015
            continue
        # Download and process the existing data
        else:
            url = ipeds_url + prefix + str(year) + suffix + '.zip'
            zip = tmp_dir + prefix + str(year) + suffix + '.zip'
            file = prefix.lower() + str(year) + suffix.lower() + '.csv'
            stream = r.get(url, stream=True)    
            print(str(stream.status_code) + '\t' + url)
            if stream.status_code == 200:
                with open(zip, 'wb') as out_file:
                    shutil.copyfileobj(stream.raw, out_file)
                z = ZipFile(zip).open(file)
                ZipFile(zip).extract(file, tmp_dir)
                enc_result = chardet.detect(z.read())
                df[prefix + suffix] = pd.read_csv(tmp_dir + '/' + file, encoding=enc_result['encoding'])

        # Eliminate and calculate columns here
    
    # Append columns into working year (add year here)
    
    # And initiate or merge to master frame
#     if year == start:
#         master = 
#     else
#         pd.concat(ipeds, merged, ignore_index=True)
    

        
        

SyntaxError: invalid syntax (<ipython-input-6-d067226bc268>, line 8)

In [14]:
# # Dataframes to be populated and merged per year: hd_df, ic_df, icay_df, efdist_df, salis_df, om_df

# base_url = 'https://nces.ed.gov/ipeds/datacenter/data/'

# for year in range(start, end):
#     # HD - Institutional Characteristics - Directory information
#     url = base_url + 'HD' + str(year) + '.zip'
#     zip = tmp_dir + 'HD' + str(year) + '.zip'
#     file = 'hd' + str(year) + '.csv'
#     stream = r.get(url, stream=True)
#     if stream.status_code == 200:
#         with open(zip, 'wb') as out_file:
#             shutil.copyfileobj(stream.raw, out_file)
#         z = ZipFile(zip).open(file)
#         ZipFile(zip).extract(file, tmp_dir)
#         enc_result = chardet.detect(z.read())
#         hd_df = pd.read_csv(tmp_dir + '/' + file, encoding=enc_result['encoding'])

#     # IC - Institutional Characteristics - Educational offerings, organization, services and athletic associations
#     url = base_url + 'IC' + str(year) + '.zip'
#     zip = tmp_dir + 'IC' + str(year) + '.zip'
#     file = 'ic' + str(year) + '.csv'
#     stream = r.get(url, stream=True)
#     if stream.status_code == 200:
#         with open(zip, 'wb') as out_file:
#             shutil.copyfileobj(stream.raw, out_file)
#         z = ZipFile(zip).open(file)
#         ZipFile(zip).extract(file, tmp_dir)
#         enc_result = chardet.detect(z.read())
#         ic_df = pd.read_csv(tmp_dir + '/' + file, encoding=enc_result['encoding'])

#     # ICAY - Institutional Characteristics	Student charges for academic year programs
#     url = base_url + 'IC' + str(year) + '_AY.zip'
#     zip = tmp_dir + 'IC' + str(year) + 'AY.zip'
#     file = 'ic' + str(year) + '_ay.csv'
#     stream = r.get(url, stream=True)
#     if stream.status_code == 200:
#         with open(zip, 'wb') as out_file:
#             shutil.copyfileobj(stream.raw, out_file)
#         z = ZipFile(zip).open(file)
#         ZipFile(zip).extract(file, tmp_dir)
#         enc_result = chardet.detect(z.read())
#         hd_df = pd.read_csv(tmp_dir + '/' + file, encoding=enc_result['encoding'])

#     # EFDIST - Fall Enrollment - Distance education status and level of student: Fall 2017
#     url = base_url + 'EF' + str(year) + 'A_DIST.zip'
#     zip = tmp_dir + 'EF' + str(year) + 'A_DIST.zip'
#     file = 'ef' + str(year) + 'a_dist.csv'
#     stream = r.get(url, stream=True)
#     if stream.status_code == 200:
#         with open(zip, 'wb') as out_file:
#             shutil.copyfileobj(stream.raw, out_file)
#         z = ZipFile(zip).open(file)
#         ZipFile(zip).extract(file, tmp_dir)
#         enc_result = chardet.detect(z.read())
#         hd_df = pd.read_csv(tmp_dir + '/' + file, encoding=enc_result['encoding'])

#     # SAL_IS - Instructional Staff/Salaries - Number and salary outlays for full-time nonmedical instructional staff, by gender, and academic rank
#     url = base_url + 'SAL' + str(year) + '_IS.zip'
#     zip = tmp_dir + 'SAL' + str(year) + '_IS.zip'
#     file = 'sal' + str(year) + '_is.csv'
#     stream = r.get(url, stream=True)
#     if stream.status_code == 200:
#         with open(zip, 'wb') as out_file:
#             shutil.copyfileobj(stream.raw, out_file)
#         z = ZipFile(zip).open(file)
#         ZipFile(zip).extract(file, tmp_dir)
#         enc_result = chardet.detect(z.read())
#         hd_df = pd.read_csv(tmp_dir + '/' + file, encoding=enc_result['encoding'])

#     # OM - Outcome Measures - Award and enrollment data at four, six and eight years of entering degree/certificate-seeking undergraduate cohorts
#     url = base_url + 'OM' + str(year) + '.zip'
#     zip = tmp_dir + 'OM' + str(year) + '.zip'
#     file = 'om' + str(year) + '.csv'
#     stream = r.get(url, stream=True)
#     if stream.status_code == 200:
#         with open(zip, 'wb') as out_file:
#             shutil.copyfileobj(stream.raw, out_file)
#         z = ZipFile(zip).open(file)
#         ZipFile(zip).extract(file, tmp_dir)
#         enc_result = chardet.detect(z.read())
#         hd_df = pd.read_csv(tmp_dir + '/' + file, encoding=enc_result['encoding'])
        
        

In [ ]:
if not dev:
    shutil.rmtree(tmp_dir)